In [1]:
import task_reader as tr
import os
import pandas as pd
import copy
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import numpy as np
import math
%load_ext autoreload
%autoreload 2

In [2]:
def walk_through_files(path, file_extension='.md'):
    for (dirpath, dirnames, filenames) in os.walk(path):
        for filename in filenames:
            if filename.endswith(file_extension):
                yield os.path.join(dirpath, filename)

In [40]:
### We have to make it not exclusive to a particular date
path = "../../Accountability/Dailies/"
dir_list = os.listdir(path)

tasks = []      
for fname in walk_through_files(path):
    date = os.path.basename(fname).split('.')[0]
    task_mode = fname.split('/')[-2]
    print(date)
    note_details = tr.task_features(date, task_mode, fname)
#     print(note_details)
    tasks.append(note_details.copy())


df = pd.DataFrame.from_records(tasks)
df.sort_values('date').head(len(tasks))        

2023-11-01
2023-10-02
2023-10-10
2023-10-25
2023-10-02
2023-10-26
2023-09-16
2023-10-23
2023-10-11
2023-10-25
2023-10-15
2023-10-02
2023-09-16
2023-10-16
2023-10-13
2023-10-09
2023-11-01
2023-10-10
2023-10-11
2023-10-25
can't do that
2023-10-05
2023-09-15


,date,task_mode,vd,d,n,p,vp,work_time,elapsed_time,breaks,productivity_pulse
21,2023-09-15,Development_Mode,0,130,165,70,0,365,395,1,45.890411
6,2023-09-16,Job_Mode,0,90,0,63,0,153,180,2,45.588235
12,2023-09-16,Development_Mode,0,0,0,46,173,219,260,1,94.748858
1,2023-10-02,Learning_Mode,0,0,69,0,0,69,69,0,50.000000
4,2023-10-02,Job_Mode,40,0,23,0,0,63,63,0,34.126984
11,2023-10-02,Development_Mode,0,0,0,36,0,36,36,0,75.000000
20,2023-10-05,Development_Mode,0,0,41,102,36,179,554,4,74.301676
15,2023-10-09,Development_Mode,32,0,31,24,42,129,374,2,64.728682
2,2023-10-10,Learning_Mode,0,28,0,0,0,28,28,0,25.000000
17,2023-10-10,Development_Mode,0,0,0,46,33,79,140,1,85.443038


In [41]:
dailies_avg = df.groupby('date').mean(numeric_only=True)
dailies_avg.head()
dailies_avg.plot.line(y='productivity_pulse')
# figs, axs = subplots()
# df['productivity_pulse'].plot.line(ax=)

<Axes: xlabel='date'>

In [42]:
#  %matplotlib

Using matplotlib backend: MacOSX


Traceback (most recent call last):
  File "/Users/danielvarzari/anaconda3/lib/python3.11/site-packages/matplotlib/backend_bases.py", line 1226, in _on_timer
    ret = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danielvarzari/anaconda3/lib/python3.11/site-packages/matplotlib/backends/backend_macosx.py", line 68, in callback_func
    callback()
  File "/Users/danielvarzari/anaconda3/lib/python3.11/site-packages/matplotlib/backends/backend_macosx.py", line 88, in _draw_idle
    self.draw()
  File "/Users/danielvarzari/anaconda3/lib/python3.11/site-packages/matplotlib/backends/backend_macosx.py", line 50, in draw
    super().draw()
  File "/Users/danielvarzari/anaconda3/lib/python3.11/site-packages/matplotlib/backends/backend_agg.py", line 400, in draw
    self.figure.draw(self.renderer)
  File "/Users/danielvarzari/anaconda3/lib/python3.11/site-packages/matplotlib/artist.py", line 95, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
           

## Grouping by Date and Graphing Task Progression
Now we have to retrieve the 

In [43]:
# Choosing the data we care about
ri = df[['date', 'productivity_pulse', 'task_mode', 'work_time']].sort_values('date')
mask = ~ri.task_mode.isin(['Job_Mode', 'Development_Mode', 'Learning_Mode'])
relevant_info = ri.drop(ri[mask].index)

# date sorting
relevant_info = relevant_info.set_index(pd.DatetimeIndex(relevant_info['date'].values))
grouped_ri = relevant_info.groupby('task_mode')

fig, ax1 = plt.subplots(figsize = (12,6))
# setting up two axis for the plots
# Rotate and align the tick labels so they look better.
fig.autofmt_xdate()
ax2 = ax1.twinx()
date_bins = np.arange(0, len(relevant_info['date'].unique()))
ax1.xaxis.set_major_locator(tick.FixedLocator(date_bins)) 


# bar graph variables
offset_count = 0
offset = 0

for name, group in grouped_ri:
    group = group.sort_values('date') #sorting data by date -> sequential
    dates = group['date'] # x axis data

    # productivity pulse data & labels
    productivity_pulse = group['productivity_pulse'] # y axis data
    task = name.split()[0].split('_')[0]
    label1 = task + ' Productivity '  # grouped by task so this is naturally the first value in the tuple
    
    # values for bar indices
    ind = np.array([])
    widths = np.array([])
    offsets = np.array([])
    for d in group['date']:
        # getting the index with respect to the entire date list
        donde = np.where(relevant_info['date'].unique() == d)
        ind = np.append(ind, donde[0][0])
    
        # the width size for each bar
        number_of_tasks_on_date = len(relevant_info[relevant_info['date'] == d])
        size = .9 / number_of_tasks_on_date
        widths = np.append(widths, size)
        
        #affecting the offset size
        if number_of_tasks_on_date == 1:
            offset = 0
        elif number_of_tasks_on_date == 2:
            offset = min(offset_count, 1) * size + -.25
        elif number_of_tasks_on_date == 3:
            offset = offset_count * size + -size
        offsets = np.append(offsets, offset)
    
    # productivity plot
    ax1.plot(ind, productivity_pulse, label=label1)
    ax1.set_ylabel('Productivity Pulse (%)')
    ax1.set_xlabel('Date')
    
    # work time plot
    ax2.set_ylabel('Work Time (in min)')
    work_time = group['work_time']
    name = task + ' Time'
    width = .3
    ax2.bar(ind + offsets, work_time, alpha=.2, align='center', width=widths, label=name)
    offset_count += 1
    
    
# giving the work time a common denominator so the yaxis grids line up
bound = int(ax2.get_ylim()[1])
bound -= bound % -100
n = math.gcd(math.gcd(10, bound), 10) + 1
ax1.set_ylim(0,100)
ax1.set_yticks(np.linspace(0,100, n))
ax2.set_yticks(np.linspace(0, bound, n))

# x labels by date instead of index
ax2.set_xticklabels(relevant_info['date'].unique())

# showing plot
plt.title('Productivity Pulse Over Time')
fig.legend(loc='upper right')
plt.grid(True)
plt.savefig('productivity.png')

In [7]:
'''
When I eventually group by date this will be a useful manager

date_manager = {
    "date": None,
    "notes_analyzed": 0,
    "size": .8
}


curr = name.split("-")[-1]
    if date_manager["date"] != curr:
        date_manager["date"] = curr
        date_manager["notes_analyzed"] = 0
    else:
        date_manager["size"] = date_manager["size"] * .5
'''

'\nWhen I eventually group by date this will be a useful manager\n\ndate_manager = {\n    "date": None,\n    "notes_analyzed": 0,\n    "size": .8\n}\n\n\ncurr = name.split("-")[-1]\n    if date_manager["date"] != curr:\n        date_manager["date"] = curr\n        date_manager["notes_analyzed"] = 0\n    else:\n        date_manager["size"] = date_manager["size"] * .5\n'